In [1]:
from mig import MIG
import pandas as pd
import numpy as np

## CGVAE

In [27]:
df = pd.read_csv('data/cgvae_latent_vars.csv')

In [10]:
df.head()

,m,n,z0_mean,z1_mean,z2_mean,z3_mean,z4_mean,z0_logvar,z1_logvar,z2_logvar,z3_logvar,z4_logvar,node,graphid
0,2,10,-0.000559,0.032740,0.037001,0.003759,0.018324,0.012386,-0.002225,0.007677,-0.018260,0.023678,node0,11931
1,2,10,0.001921,0.051241,0.080356,0.017393,0.036365,0.036735,-0.009989,-0.001007,-0.045550,0.037277,node1,11931
2,2,10,0.001358,0.059037,0.078070,0.015362,0.039275,0.031837,-0.008589,0.002799,-0.042603,0.046334,node2,11931
3,2,10,0.002223,0.032883,0.059734,0.010430,0.019892,0.032319,-0.002235,0.000138,-0.032970,0.023751,node3,11931
4,2,10,-0.002418,0.027271,0.035522,0.004417,0.018304,0.012084,-0.005383,0.008458,-0.018064,0.017135,node4,11931


In [11]:
v1 = np.concatenate(df.groupby("graphid")["n"].unique().to_numpy())
v2 = np.concatenate(df.groupby("graphid")["m"].unique().to_numpy())
v = np.vstack((v1, v2))
v1 = v1.reshape((2304, 1))
v2 = v2.reshape((2304, 1))

In [12]:
def MIG_node(node, data, v1, v2):
    z = data[data["node"] == node].iloc[:, 2:7].to_numpy().T
    total_score = MIG(z, v)
    n_score = MIG(z, v1.T)
    m_score = MIG(z, v2.T)
    print(
        f"Node: {node},\nTotal MIG: {total_score.compute_mig()},\nMIG for n:{n_score.compute_mig()},\nMIG for m/n:{m_score.compute_mig()}"
    )
for i in range(6):
    node = 'node'+str(i)
    MIG_node(node,df,v1,v2)

Node: node0,
Total MIG: 0.001545221023340999,
MIG for n:0.0003835806734048586,
MIG for m/n:0.0027068613732771397
Node: node1,
Total MIG: 0.0018768180841123164,
MIG for n:0.0010607141304280416,
MIG for m/n:0.0026929220377965913
Node: node2,
Total MIG: 0.0009178260158507376,
MIG for n:0.0007791273914822781,
MIG for m/n:0.001056524640219197
Node: node3,
Total MIG: 0.0003077512068311433,
MIG for n:0.00012351203444958304,
MIG for m/n:0.0004919903792127035
Node: node4,
Total MIG: 0.0014989323201515395,
MIG for n:0.0006829218778696873,
MIG for m/n:0.002314942762433392
Node: node5,
Total MIG: 0.001107041339131845,
MIG for n:0.0008947713226648222,
MIG for m/n:0.0013193113555988677


In [13]:
z = df.iloc[:, 2:7].to_numpy().T
v1 = df["n"].to_numpy()
v2 = df["m"].to_numpy()
v = np.vstack((v1, v2))
v1 = v1.reshape((20736, 1))
v2 = v2.reshape((20736, 1))
print(
    f"total:{MIG(z, v).compute_mig()},\n n:{MIG(z, v1.T).compute_mig()},\n m:{MIG(z, v2.T).compute_mig()}"
)

total:0.06507099383652612,
 n:0.02305838598104319,
 m:0.10708360169200905


## LSTM

### Linear BA m

In [14]:
zlstm = pd.read_csv("data/vae_latent_scenario_3.csv", index_col=0)
vlstm = pd.read_csv("data/vae_params_scenario_3.csv", index_col=0)
zlstm.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.598955,-0.806814,1.518124,1.073263,-0.789962,-1.571995,0.785124,-1.503613,0.424778,-0.799481,...,0.440538,-0.717760,-2.710367,-0.846979,0.314923,-1.396957,1.172346,-1.389031,1.121538,-1.889559
1,0.286188,-0.068022,-0.595867,0.329084,-0.427043,2.186117,0.039445,-0.576465,0.994669,-1.146221,...,0.634108,0.242836,-0.753537,-0.933799,-0.499300,0.052821,-1.235559,0.077060,0.330694,-0.298524
2,-0.186810,-0.438212,0.745647,-0.330436,-2.416354,2.024314,1.568221,0.120011,0.936649,0.729429,...,-3.316480,-1.067274,1.183420,1.328951,2.248298,-1.134372,-0.128524,0.520403,0.205554,1.001499


In [15]:
vlstm.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,7.0,9.0,3.0,5.0,8.0,9.0,5.0,10.0,5.0,7.0,...,6.0,8.0,9.0,7.0,5.0,8.0,3.0,8.0,4.0,10.0
1,4.0,8.0,2.0,2.0,7.0,8.0,1.0,8.0,3.0,1.0,...,1.0,7.0,2.0,1.0,3.0,6.0,1.0,7.0,3.0,4.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Third row in `vlstm` corresponds to the $\alpha$ values. Since we are in the linear case, $\alpha$ is always 1. We cannot compute the MIG score considering $\alpha$ in this case, since it does not vary and hence the entropy is zero. Therefore, we remove it from our data.

In [16]:
vlstm = vlstm.drop([2])

In [17]:
vlstm_array = vlstm.to_numpy()
MIG(zlstm, vlstm_array).compute_mig()

C:\Users\Alacrity\Documents\GitHub\graphDisentanglement\submission\mig.py:29: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  return (data - mean[:, np.newaxis]) / stddev[:, np.newaxis], mean, stddev


0.25920836177437057

### Linear BA m/n

In [18]:
#Define m/n and add it to our array
vlstm = vlstm.append(vlstm.apply(lambda x: x[1] / x[0], axis=0), ignore_index=True)
#remove m
vlstm = vlstm.drop([1])
vlstm_array = vlstm.to_numpy()
MIG(zlstm, vlstm_array).compute_mig()

C:\Users\Alacrity\Documents\GitHub\graphDisentanglement\submission\mig.py:29: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  return (data - mean[:, np.newaxis]) / stddev[:, np.newaxis], mean, stddev


0.2851049639682969

### Non-linear BA

In [20]:
zlstm = pd.read_csv("data/vae_latent_scenario_4.csv", index_col=0)
vlstm = pd.read_csv("data/vae_params_scenario_4.csv", index_col=0)
zlstm.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-1.200958,-0.748129,-1.609783,1.534683,-0.447586,0.497946,-0.316533,1.335989,-0.726742,-1.826400,...,-1.094135,-1.017225,1.081904,-1.871739,-0.963772,-0.392770,-0.928829,-1.076690,-0.459370,0.841868
1,-0.879703,-0.746058,-1.421416,-1.594064,-0.210879,-1.168217,-1.665984,3.290594,0.255643,-1.656996,...,-0.113013,0.026096,-1.270595,-0.944150,-1.202214,-0.833773,-0.470943,-1.128723,-1.052766,-1.321458
2,0.845625,1.168285,1.468284,-0.045874,1.223662,-0.297803,-0.609097,-0.229409,0.625650,1.631372,...,1.213277,1.801117,-0.636058,1.980445,0.576992,0.671615,1.025956,0.921024,-0.397102,-2.042325


In [21]:
vlstm.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,9.000000,8.000000,10.000000,4.000000,6.000000,5.000000,7.000000,3.00000,7.000000,10.000000,...,9.000000,8.000000,5.000000,9.000000,8.000000,6.000000,8.000000,9.000000,7.000000,5.000000
1,3.000000,6.000000,8.000000,1.000000,5.000000,4.000000,3.000000,2.00000,4.000000,6.000000,...,4.000000,3.000000,3.000000,4.000000,2.000000,1.000000,7.000000,2.000000,1.000000,3.000000
2,0.219958,0.602079,0.494748,0.859589,0.496658,0.496658,0.898035,0.71054,0.047038,0.337226,...,0.633313,0.451392,0.587359,0.530978,0.905795,0.587359,0.951938,0.633313,0.244185,0.716113


### with m

In [22]:
vlstm_array = vlstm.to_numpy()
MIG(zlstm, vlstm_array).compute_mig()

C:\Users\Alacrity\Documents\GitHub\graphDisentanglement\submission\mig.py:29: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  return (data - mean[:, np.newaxis]) / stddev[:, np.newaxis], mean, stddev


0.11687011349363031

### with m/n

In [23]:
#Define m/n and add it to our array
vlstm = vlstm.append(vlstm.apply(lambda x: x[1] / x[0], axis=0), ignore_index=True)
#remove m
vlstm = vlstm.drop([1])
vlstm_array = vlstm.to_numpy()
MIG(zlstm, vlstm_array).compute_mig()

C:\Users\Alacrity\Documents\GitHub\graphDisentanglement\submission\mig.py:29: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  return (data - mean[:, np.newaxis]) / stddev[:, np.newaxis], mean, stddev


0.13107513831698445